In [8]:
import os
import pandas as pd
import csv

In [9]:
# TIMIT 데이터가 저장된 루트 디렉토리 경로
timit_root = "/home/woongjae/noise-tracing/Datasets/TIMIT"  # 여기를 실제 경로로 변경하세요
output_csv = "meta_timit_ori.csv"

# 모든 .WAV 파일 경로를 리스트로 수집
wav_file_paths = []
for root, dirs, files in os.walk(timit_root):
    for file in files:
        if file.lower().endswith(".wav"):
            full_path = os.path.join(root, file)
            wav_file_paths.append(full_path)

# CSV로 저장
with open(output_csv, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["file_path"])  # 헤더
    for path in wav_file_paths:
        writer.writerow([path])

print(f"{len(wav_file_paths)}개의 .WAV 파일 경로가 '{output_csv}'에 저장되었습니다.")

6300개의 .WAV 파일 경로가 'meta_timit_ori.csv'에 저장되었습니다.


In [2]:
import os
import pandas as pd

# 경로 설정
meta_csv_path = "/home/woongjae/noise-tracing/new_dataset/Dataset/metafile/meta_clean.csv"  # 기존 메타
timit_root = "/home/woongjae/noise-tracing/Datasets/TIMIT"  # TIMIT 원본 위치
output_csv_path = "/home/woongjae/noise-tracing/new_dataset/Dataset/metafile/meta_clean_updated.csv"  # 저장할 메타

# 기존 메타 불러오기
df = pd.read_csv(meta_csv_path)

# CommonVoice 제거
df = df[df["Group"] != "TIMIT"]

# TIMIT 수집
new_rows = []
for root, dirs, files in os.walk(timit_root):
    for file in files:
        if file.lower().endswith(".wav"):
            full_path = os.path.join(root, file)

            original_fname = os.path.splitext(file)[0]
            speaker_id = os.path.basename(os.path.dirname(full_path))
            utt = f"{speaker_id}_{original_fname}"  # 중복 방지용 utt

            new_rows.append({
                "File_path": full_path,                      # 원본 경로 그대로 (복사 전)
                "utt": utt,
                "speaker ID": speaker_id,
                "gender": "Unknown",
                "Group": "TIMIT",
                "Label2": "bonafide",
                "Label1": "clean"
            })

# 병합
df_new = pd.DataFrame(new_rows)
df_combined = pd.concat([df, df_new], ignore_index=True)

# 저장
df_combined.to_csv(output_csv_path, index=False)
print(f"✅ 총 {len(df_combined)}개의 샘플이 저장된 메타파일 생성 완료: {output_csv_path}")


✅ 총 49000개의 샘플이 저장된 메타파일 생성 완료: /home/woongjae/noise-tracing/new_dataset/Dataset/metafile/meta_clean_updated.csv


##### CommonVoice 지우기

In [6]:
import os
import pandas as pd

# 설정
meta_dir = "/home/woongjae/noise-tracing/new_dataset/Dataset/metafile"  # 여기를 실제 경로로 수정하세요
meta_all_path = "/home/woongjae/noise-tracing/new_dataset/Dataset/meta_ori/meta_clean.csv"
output_dir = os.path.join(meta_dir, "meta_clean")
os.makedirs(output_dir, exist_ok=True)

### 1. meta_all.csv에서 CommonVoice 오디오 파일 삭제만 수행 (메타는 그대로 유지)
df_all = pd.read_csv(meta_all_path)
df_cv = df_all[df_all["Group"] == "TIMIT"]
delete_paths = df_cv["File_path"].dropna().tolist()
deleted = 0

for path in delete_paths:
    try:
        os.remove(path)
        deleted += 1
    except FileNotFoundError:
        print(f"[경고] 파일 없음: {path}")
    except Exception as e:
        print(f"[에러] {path}: {e}")

print(f"[1] meta_all.csv 기준으로 CommonVoice 오디오 {deleted}개 삭제 완료 (메타는 유지됨)")

### 2. 모든 meta_*.csv 파일에서 CommonVoice 행 제거
meta_files = [f for f in os.listdir(meta_dir)
              if f.startswith("meta_") and f.endswith(".csv") and f != "meta_all.csv"]

for file in meta_files:
    file_path = os.path.join(meta_dir, file)
    df = pd.read_csv(file_path)

    if "Group" not in df.columns:
        print(f"[스킵] {file}: 'Group' 컬럼 없음")
        continue

    before = len(df)
    df_cleaned = df[df["Group"] != "TIMIT"]
    after = len(df_cleaned)

    # 저장
    cleaned_path = os.path.join(output_dir, file)
    df_cleaned.to_csv(cleaned_path, index=False)
    print(f"[2] {file}: {before}개 중 {before - after}개 CommonVoice 제거 → {after}개 저장됨")


[경고] 파일 없음: /home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean/SX413.wav
[경고] 파일 없음: /home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean/SX143.wav
[경고] 파일 없음: /home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean/SX53.wav
[경고] 파일 없음: /home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean/SI1043.wav
[경고] 파일 없음: /home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean/SX323.wav
[경고] 파일 없음: /home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean/SA2.wav
[경고] 파일 없음: /home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean/SX233.wav
[경고] 파일 없음: /home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean/SI2303.wav
[경고] 파일 없음: /home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean/SA1.wav
[경고] 파일 없음: /home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean/SI1673.wav
[경고] 파일 없음: /home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean/SX413.wav
[경고] 파일 없음: /home/woongjae/noise-tracing/new_dataset/Dat

In [3]:
import os
import shutil
import pandas as pd

# 경로 설정
meta_path = "/home/woongjae/noise-tracing/new_dataset/Dataset/metafile/meta_clean_updated.csv"
dst_dir = "/home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean"
os.makedirs(dst_dir, exist_ok=True)

# 메타데이터 불러오기
df = pd.read_csv(meta_path)

# TIMIT 행만 필터링
df_timit = df[df["Group"] == "TIMIT"].copy()

# 파일 복사 및 경로 갱신
new_paths = []

for _, row in df_timit.iterrows():
    src_path = row["File_path"]
    utt = row["utt"]
    new_fname = f"{utt}.wav"  # 이미 speakerID_파일명 형식
    dst_path = os.path.join(dst_dir, new_fname)

    try:
        shutil.copy2(src_path, dst_path)
    except Exception as e:
        print(f"[ERROR] {src_path} → {dst_path}: {e}")

    new_paths.append(dst_path)

# 복사 후 경로 갱신된 메타 저장 (선택 사항)
df_timit["File_path"] = new_paths
df_timit.to_csv(meta_path.replace(".csv", "_copied.csv"), index=False)
print(f"✅ TIMIT 파일 {len(df_timit)}개 복사 완료 → {dst_dir}")


✅ TIMIT 파일 6300개 복사 완료 → /home/woongjae/noise-tracing/new_dataset/Dataset/Bonafide/Clean


In [4]:
import pandas as pd

# 기존 (TIMIT 제외) clean 메타파일 경로
old_clean_path = "/home/woongjae/noise-tracing/new_dataset/Dataset/metafile/meta_clean.csv"  # 기존 clean 전체 메타
new_timit_clean_path = "/home/woongjae/noise-tracing/new_dataset/Dataset/metafile/meta_clean_updated_copied.csv"

# 최종 저장 위치
final_clean_path = "/home/woongjae/noise-tracing/new_dataset/Dataset/metafile/meta_clean_final.csv"

# 불러오기
df_old = pd.read_csv(old_clean_path)
df_timit = pd.read_csv(new_timit_clean_path)

# 기존 clean 중 TIMIT 아닌 샘플만 추출
df_old_filtered = df_old[df_old["Group"] != "TIMIT"]

# 병합
df_final = pd.concat([df_old_filtered, df_timit], ignore_index=True)

# 저장
df_final.to_csv(final_clean_path, index=False)
print(f"✅ 최종 clean 메타파일 생성 완료: {final_clean_path} (총 {len(df_final)}개)")


✅ 최종 clean 메타파일 생성 완료: /home/woongjae/noise-tracing/new_dataset/Dataset/metafile/meta_clean_final.csv (총 49000개)
